In [1]:
%pip install mysql-connector-python

   ---------------------------------------- 0.0/14.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.3 MB 435.7 kB/s eta 0:00:33
   ---------------------------------------- 0.2/14.3 MB 1.5 MB/s eta 0:00:10
   ----- ---------------------------------- 2.1/14.3 MB 15.0 MB/s eta 0:00:01
   ------------------ --------------------- 6.7/14.3 MB 35.6 MB/s eta 0:00:01
   ------------------------------- -------- 11.2/14.3 MB 93.9 MB/s eta 0:00:01
   ---------------------------------------  14.3/14.3 MB 110.0 MB/s eta 0:00:01
   ---------------------------------------- 14.3/14.3 MB 81.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [56]:
import mysql.connector
from mysql.connector import Error

def create_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            port=3307,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        if connection.is_connected():
            print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")
    
    return connection

# 데이터베이스 연결 예시
connection = create_connection("project-db-cgi.smhrd.com", "moai", "smhrd1234!", "moai")

Connection to MySQL DB successful


In [28]:
import psycopg2
from psycopg2 import pool

In [33]:
dbcp = psycopg2.pool.SimpleConnectionPool(1, 20,
    user='postgres.vpcdvbdktvvzrvjfyyzm',
    password='Odvv8E1iChKjwai4',
    host='aws-0-ap-southeast-1.pooler.supabase.com',
    port=6543,
    dbname='postgres'
)

In [52]:
dbcp.putconn(db)

In [34]:
db = dbcp.getconn()

In [35]:
cur = db.cursor()

In [48]:
cur.execute('select * from langchain_pg_embedding')

In [49]:
a = cur.fetchall()

In [50]:
a

[('1ae9ef17-9a88-4de7-b325-bf1891db7df8',
  '083b048b-0d75-465f-b098-d2d5f5b0920a',
  '[-0.019805599,-0.036790576,0.0006617581,-0.019823842,-0.01739032,-0.05119121,-0.05147993,0.021056434,0.0062744045,-0.019154254,-0.0060614944,-0.016790194,-0.014545735,0.025643088,-0.013342926,0.017992174,0.008523403,0.009871156,0.012797267,-0.030628797,-0.02646202,0.06463612,0.03287555,-0.025238493,0.0038599886,-0.0058573266,-0.030974329,-0.024449704,0.03913651,-0.032354545,-0.048520055,-0.029636038,-0.04394585,-0.011818207,-0.032826014,-0.008498891,-0.05779559,-0.024725,-0.054667804,0.045037944,-0.008576954,0.000103147744,0.0061235856,-0.042072374,0.02080201,-0.034413822,-0.0024623792,-0.019966329,0.009521572,-0.031497817,-0.007311593,0.020080265,0.047810186,-0.017913554,-0.050500542,0.044242032,0.011528338,0.018724762,-0.039058145,-0.030767078,-0.025938055,0.018173153,-0.061997127,0.033824936,0.026127608,-0.020533333,-0.020511148,0.009027103,-0.028966287,-0.03699351,0.009906613,0.050971553,0.001504

In [40]:
db.commit()

In [1]:
from pypdf import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from docx import Document
from flask import *
from flask_cors import CORS
import json
import os
import sys
import datetime
from langchain_postgres import PGVector
from langchain_postgres.vectorstores import PGVector
import psycopg2
from psycopg2 import pool

In [23]:
dbcp = psycopg2.pool.SimpleConnectionPool(1, 20,
    user='postgres.vpcdvbdktvvzrvjfyyzm',
    password='Odvv8E1iChKjwai4',
    host='aws-0-ap-southeast-1.pooler.supabase.com',
    port=6543,
    dbname='postgres'
)

embedding_model = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)
def db_crawler(user_id):
    db = dbcp.getconn()
    cur = db.cursor()
    cur.execute("""
                select cl.chat_at, u.user_name, cl.cl_chat
                from chat_logs cl join users u on cl.user_id = u.user_id
                where cl_type = 'text' and room_id in (select room_id from room_users where user_id = %s)
                order by chat_at
                """,
                (user_id,))
    chat_rows = cur.fetchall()
    cur.execute("""
                select cal_title, cal_start_date, cal_end_date, cal_location
                from calendars
                where user_id = %s or cal_id in (select cal_sub_id from calendar_shared)
                order by cal_start_date
                """,
                (user_id,))
    cal_rows = cur.fetchall()
    dbcp.putconn(db)

    db_crawls = ''

    for row in chat_rows:
        _ = '\n'
        db_crawls += f"{row[0].strftime('%G-%m-%d %T')}에 {row[1]}이(가) '{row[2].replace(_, '')}'라 말함. "
    for row in cal_rows:
        db_crawls += f"{row[1].replace('T', ' ')}부터 {row[2].replace('T', ' ')}까지 일정: '{row[0]}'이 {row[3] +'에서 ' if row[3] else ''}있습니다. "
    
    
    db_crawls += datetime.datetime.now().strftime('오늘은 %G-%m-%d, %A입니다.')
    db_split = text_splitter.split_text(db_crawls)
    
    return db_split

llm_model = ChatOllama(
    model='meta-llama-3.1',
    base_url='https://b1f9-35-197-57-104.ngrok-free.app'      # 주석 해제 시 코랩 자원으로 돌아감, # 주석 설정 시 로컬 자원으로 돌아감 
)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

prompt_template = '''about question, answer me only use korean.
if you can't understand the question, you can use the following pieces of context to answer.

context: {chat_history}

Question: {question}
Helpful Answer:'''
# rag_prompt = PromptTemplate.from_template(prompt_template)
chat_prompt = PromptTemplate(template=prompt_template, input_variables=['chat_history', 'question'])
splits = db_crawler('user_id')
vectorstore = FAISS.from_texts(splits, embedding_model)

In [27]:
memory = ConversationBufferMemory(
    memory_key='chat_history',
    return_messages=True
)
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm_model,
    retriever=vectorstore.as_retriever(),
    condense_question_prompt=chat_prompt,
    memory=memory
)

memory

ConversationBufferMemory(return_messages=True, memory_key='chat_history')

In [29]:
res = conversation_chain({'question': '내 이름을 기억해?'})
res

{'question': '내 이름을 기억해?',
 'chat_history': [HumanMessage(content='내 이름은 박건이야'),
  AIMessage(content='박건이라고요'),
  HumanMessage(content='내 이름을 기억해?'),
  AIMessage(content='박건이라고 기억합니다')],
 'answer': '박건이라고 기억합니다'}

In [21]:
memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='hello'), AIMessage(content='안녕하세요')]), return_messages=True, memory_key='chat_history')